# Tutorial Goal

This tutorial aims to show how to **configure and run** a predefined set of
**synthetic workload** using the **executor module** provided by LISA.

# Configure logging

In [2]:
import logging
reload(logging)
log_fmt = '%(asctime)-9s %(levelname)-8s: %(message)s'
logging.basicConfig(format=log_fmt)

In [3]:
# Execute this cell to enabled executor debugging statements
logging.getLogger().setLevel(logging.DEBUG)

# Target Configuration

In [4]:
# Setup a target configuration
my_target_conf = {
    
    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'juno2',
    
    # Target board IP/MAC address
    "host"        : '10.1.207.30',
    
    # Login credentials
    "username"    : 'root',
    "password"    : 'password',
    
    "rtapp-calib" : {
        "0": 320, "1": 124, "2": 125, "3": 318, "4": 316, "5": 317
    },
}

# Tests Configuration

In [5]:
my_tests_conf = {

    # Folder where all the results will be collected
    "results_dir" : "ExecutorExample",

    # Platform configurations to test
    "confs" : [
        {
            "tag"            : "base",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "NO_ENERGY_AWARE",  # Disable EAS
            "cpufreq"        : {                   # Use PERFORMANCE CpuFreq
                "governor" : "performance",
            },
        },
        {
            "tag"            : "eas",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "ENERGY_AWARE",     # Enable EAS
            "cpufreq"        : {                   # Use PERFORMANCE CpuFreq
                "governor" : "performance",
            },
        },
    ],
    
    # Workloads to run (on each platform configuration)
    "wloads" : {
        # Run hackbench with 1 group using pipes
        "perf" : {
            "type" : "perf_bench",
            "conf" : {
                "class" : "messaging",
                "params" : {
                    "group" :    1,
                    "loop"  :   10,
                    "pipe"  : True,
                    "thread": True,
                }
            }
        },
        # Run a 20% duty-cycle periodic task
        "rta" : {
            "type" : "rt-app",
            "loadref" : "big",
            "conf" : {
                "class"  : "profile",
                "params"  : {
                    "p20" : {
                        "kind"   : "Periodic",
                        "params" : {
                            "duty_cycle_pct" : 20,
                         },
                    },
                },
            },
        },
    },
    
    # Number of iterations for each workload
    "iterations" : 1,
    
    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "cpu_frequency",
         ],
         "buffsize" : 80 * 1024,
    },
    
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'perf', 'rt-app' ],
    
    # Modules required by these experiments
    "modules"     : [ 'bl', 'cpufreq' ],

}

# Tests execution

In [6]:
from executor import Executor

executor = Executor(my_target_conf, my_tests_conf)

2016-04-27 15:48:33,512 INFO    :         Target - Loading custom (inline) test configuration
2016-04-27 15:48:33,514 INFO    :         Target - Using base path: /data/work/juno/lisa
2016-04-27 15:48:33,515 INFO    :         Target - Loading custom (inline) target configuration
2016-04-27 15:48:33,516 DEBUG   :         Target - Target configuration {'username': 'root', 'rtapp-calib': {'1': 124, '0': 320, '3': 318, '2': 125, '5': 317, '4': 316}, 'platform': 'linux', 'host': '10.1.207.30', 'board': 'juno2', 'password': 'password'}
2016-04-27 15:48:33,517 INFO    :         Target - Loading custom (inline) test configuration
2016-04-27 15:48:33,517 DEBUG   :         Target - Test configuration {'username': 'root', 'rtapp-calib': {'1': 124, '0': 320, '3': 318, '2': 125, '5': 317, '4': 316}, 'platform': 'linux', 'host': '10.1.207.30', 'board': 'juno2', 'password': 'password'}
2016-04-27 15:48:33,519 INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq', 'hwmon']
2016-04-27 15:4

In [ ]:
executor.run()

2016-04-27 15:49:32,702 INFO    : 
2016-04-27 15:49:32,704 INFO    : ################################################################################
2016-04-27 15:49:32,704 INFO    :       Executor - Experiments execution
2016-04-27 15:49:32,705 INFO    : ################################################################################
2016-04-27 15:49:32,706 INFO    : 
2016-04-27 15:49:32,707 INFO    : ================================================================================
2016-04-27 15:49:32,707 INFO    :   TargetConfig - configuring target for [base] experiments
2016-04-27 15:49:32,708 DEBUG   :    TargetSetup - Setup RT-App run folder [/root/devlib-target/run_dir]...
2016-04-27 15:49:34,519 INFO    :  SchedFeatures - Set scheduler feature: NO_ENERGY_AWARE
2016-04-27 15:49:34,928 INFO    :        CPUFreq - Configuring all CPUs to use [performance] governor
2016-04-27 15:49:35,834 DEBUG   :          RTApp - Configuring [profile] rt-app...
2016-04-27 15:49:35,836 INFO    :   

In [ ]:
!tree {executor.te.res_dir}